In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
num_epochs = 50
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

In [5]:
train_paths = glob('../dataset/cifar/train/*.png')[:1000]
test_paths = glob('../dataset/cifar/test/*.png')[:1000]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

## Training
- http://localhost:6006

In [17]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

# history에 꼭 담아줘야 함
history = model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs
)

Epoch 1/50
31/31 [==============================] - 8s 244ms/step - loss: 2.3126 - accuracy: 0.1219 - val_loss: 2.2948 - val_accuracy: 0.1381
Epoch 2/50
31/31 [==============================] - 9s 300ms/step - loss: 2.2638 - accuracy: 0.1581 - val_loss: 2.2140 - val_accuracy: 0.1472
Epoch 3/50
31/31 [==============================] - 10s 310ms/step - loss: 2.2545 - accuracy: 0.1581 - val_loss: 2.2346 - val_accuracy: 0.1865
Epoch 4/50
31/31 [==============================] - 10s 314ms/step - loss: 2.1185 - accuracy: 0.2035 - val_loss: 2.0636 - val_accuracy: 0.2671
Epoch 5/50
31/31 [==============================] - 10s 330ms/step - loss: 2.0930 - accuracy: 0.2076 - val_loss: 2.1755 - val_accuracy: 0.1774
Epoch 6/50
31/31 [==============================] - 10s 317ms/step - loss: 2.1143 - accuracy: 0.2298 - val_loss: 2.0250 - val_accuracy: 0.2339
Epoch 7/50
31/31 [==============================] - 9s 302ms/step - loss: 2.0723 - accuracy: 0.2362 - val_loss: 2.0682 - val_accuracy: 0.2288
Ep

## Predict

#### 이미지를 Load 직접해서 넣는 방법

In [15]:
path = test_paths[0]
path

'../dataset/cifar/test\\0_cat.png'

In [16]:
gfile = tf.io.read_file(path)
image = tf.io.decode_image(gfile, dtype=tf.float32)
image.shape

TensorShape([32, 32, 3])

In [19]:
# batch 사이즈 추가 해야함
# 4차원으로 학습했기 때문에 predict도 4차원으로 맞춰줘야 함.
image = image[tf.newaxis, ...]
image.shape

TensorShape([1, 32, 32, 3])

In [20]:
pred = model.predict(image)
pred.shape

(1, 10)

In [21]:
# 이 중에서 각 클래스 별로 젤 높은 숫자가 예측하는 숫자가 됨
pred

array([[0.0311225 , 0.02930307, 0.03574135, 0.15310898, 0.2588807 ,
        0.15171532, 0.01177117, 0.28594318, 0.01837261, 0.02404114]],
      dtype=float32)

In [22]:
np.argmax(pred)

7

## generator에서 데이터 가져오는 방법

In [23]:
test_image, test_label = next(iter(test_dataset))

In [24]:
# test는 차원 늘려줄 필요없다.
test_image.shape, test_label.shape

(TensorShape([32, 32, 32, 3]), TensorShape([32]))

In [27]:
pred = model.predict(test_image)
pred.shape

(32, 10)

In [28]:
pred[0]

array([0.03112249, 0.02930307, 0.03574135, 0.15310901, 0.25888065,
       0.15171535, 0.01177117, 0.28594318, 0.01837261, 0.02404112],
      dtype=float32)

## genertator에 넣는 방법

In [29]:
# repeat가 되는 데이터이기 때문에 1나만 가져오게 해줘야 함
pred = model.predict_generator(test_dataset.take(1))
pred.shape

# pred = model.predict_generator(test_dataset.take(2))
# pred.shape

# for image, label in test_dataset.take(1):
#     plt.imshow(image[0])
#     plt.show()

(32, 10)

In [30]:
image, label = next(iter(test_dataset))

In [31]:
# evaluate 은 image와 ,label을 같이 넣는다
evals = model.evaluate(image, label)
evals

32/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 3ms/sample - loss: 1.2735 -

[1.2734601497650146, 0.53125]